In [4]:
import pandas as pd
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet_ic
from nltk.text import TextCollection

In [23]:
nltk.download('gutenberg')
nltk.download('webtext')

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\Andy\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package webtext to
[nltk_data]     C:\Users\Andy\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\webtext.zip.


True

In [66]:
def isNaN(string):
    return string != string

In [69]:
train_df = pd.read_csv('train_raw.csv', index_col=0)
train_df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
267779,US,3387522,R2JEMKCQT9QTVM,B008SBZD5U,150118808,Kingdom Hearts HD 1.5 Remix,Video Games,4,0,0,N,Y,KH!>% RULE!,Its A pretty good game and it came on time but...,2015-02-01
267796,US,15137367,R3IM5TXYMA7DCQ,B00IXMF5CU,292308774,Terraria - Xbox 360,Video Games,5,1,2,N,Y,Five Stars,Excellent,2015-02-01
267818,US,15137367,R2PFXXB4EP23B,B00EM5UFEK,461081395,Plants vs. Zombies Garden Warfare,Video Games,4,0,0,N,Y,Four Stars,Excellent,2015-02-01
267871,US,4354770,R1A9DB178PFXLH,B004OPYLTS,245894499,Fishdom - Nintendo DS,Video Games,5,0,0,N,Y,Fun Game,"Fun game, levels get harder as you progress.",2015-02-01
267892,US,6698937,R1RS2GAXENP9LF,B00BGA9X9W,943154724,DualShock 4 Wireless Controller for PlayStatio...,Video Games,4,0,0,N,Y,Four Stars,Great controller,2015-02-01


In [75]:
train_df['review_body'] = train_df['review_body'].apply(lambda x: '' if isNaN(x) else x)
train_df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
267779,US,3387522,R2JEMKCQT9QTVM,B008SBZD5U,150118808,Kingdom Hearts HD 1.5 Remix,Video Games,4,0,0,N,Y,KH!>% RULE!,Its A pretty good game and it came on time but...,2015-02-01
267796,US,15137367,R3IM5TXYMA7DCQ,B00IXMF5CU,292308774,Terraria - Xbox 360,Video Games,5,1,2,N,Y,Five Stars,Excellent,2015-02-01
267818,US,15137367,R2PFXXB4EP23B,B00EM5UFEK,461081395,Plants vs. Zombies Garden Warfare,Video Games,4,0,0,N,Y,Four Stars,Excellent,2015-02-01
267871,US,4354770,R1A9DB178PFXLH,B004OPYLTS,245894499,Fishdom - Nintendo DS,Video Games,5,0,0,N,Y,Fun Game,"Fun game, levels get harder as you progress.",2015-02-01
267892,US,6698937,R1RS2GAXENP9LF,B00BGA9X9W,943154724,DualShock 4 Wireless Controller for PlayStatio...,Video Games,4,0,0,N,Y,Four Stars,Great controller,2015-02-01


In [76]:
corpus = TextCollection(train_df.review_body)

In [78]:
corpus.idf('Game')

4.033328746641025

In [137]:
def get_word_to_text_sim(string1, string2):
    print('Beginning Text Sim')
    num = 0
    
    print('Calculating Numerator')
    for w1 in string1.split():
        max_sim = 0
        for w2 in string2.split():
            w1_synset = wn.synsets(w1)
            w2_synset = wn.synsets(w2)
            if (len(w1_synset) == 0 or len(w2_synset) == 0):
                continue
            w1_syn = w1_synset[0]
            w2_syn = w2_synset[0]
            sim = w1_syn.wup_similarity(w2_syn)
            if (sim != None and sim > max_sim):
                max_sim = sim
        freq = corpus.idf(w1)
        num += max_sim * freq

    print('Calculating Denominator')
    denom = 0
    for w1 in string1.split():
        w1_synset = wn.synsets(w1)
        if (len(w1_synset) == 0):
            continue
        freq = corpus.idf(w1)
        denom += freq
    
    if (denom == 0):
        return 0
    
    return num / denom

In [138]:
def text_sim(string1, string2):
    return (get_word_to_text_sim(string1, string2) + get_word_to_text_sim(string2, string1)) / 2

In [139]:
s1 = "I'm going to buy some bread"
s2 = "I'm going to the store to purchase bread"
s3 = "I'm going to work to make some money"

In [140]:
print('w1: {} w2: {} sim: {}'.format(s1, s2, text_sim(s1, s2)))
print('w1: {} w2: {} sim: {}'.format(s1, s3, text_sim(s1, s3)))
print('w1: {} w2: {} sim: {}'.format(s2, s3, text_sim(s2, s3)))

Beginning Text Sim
Calculating Numerator
Calculating Denominator
Beginning Text Sim
Calculating Numerator
Calculating Denominator
w1: I'm going to buy some bread w2: I'm going to the store to purchase bread sim: 0.7826037244687303
Beginning Text Sim
Calculating Numerator
Calculating Denominator
Beginning Text Sim
Calculating Numerator
Calculating Denominator
w1: I'm going to buy some bread w2: I'm going to work to make some money sim: 0.5497105780746618
Beginning Text Sim
Calculating Numerator
Calculating Denominator
Beginning Text Sim
Calculating Numerator
Calculating Denominator
w1: I'm going to the store to purchase bread w2: I'm going to work to make some money sim: 0.44763921153577335


In [141]:
kingdom_hearts = train_df[train_df.product_parent == 150118808]['review_body']

In [142]:
terraria = train_df[train_df.product_parent == 292308774]['review_body']

In [143]:
controller = train_df[train_df.product_parent == 943154724]['review_body']

In [153]:
def product_sim(prod1, prod2):
    sim = 0
    count = 0
    for r1 in prod1:
        for r2 in prod2:
            sim += jaccard_similarity(r1, r2)
            count += 1
    
    return sim / count        

In [155]:
def jaccard_similarity(query, document):
    intersection = set(query).intersection(set(document))
    union = set(query).union(set(document))
    return len(intersection)/len(union)

In [158]:
def product_sim(prod1, prod2):
    return jaccard_similarity(" ".join(prod1.tolist()).split(), " ".join(prod2.tolist()).split())

In [159]:
print('p1: {} p2: {} sim: {}'.format('kingdom_hearts', 'terraria', product_sim(kingdom_hearts, terraria)))
print('p1: {} p2: {} sim: {}'.format('kingdom_hearts', 'controller', product_sim(kingdom_hearts, controller)))
print('p1: {} p2: {} sim: {}'.format('terraria', 'controller', product_sim(terraria, controller)))

p1: kingdom_hearts p2: terraria sim: 0.10227936879018118
p1: kingdom_hearts p2: controller sim: 0.1579828415986608
p1: terraria p2: controller sim: 0.05120630231413097
